In [ ]:
import pandas as pd 
import nltk
import re 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
# Word2vec
import gensim

Ancak, Metin gösterimi adımına geçmeden önce, önce ön işleme tabi tutulması gereken temizlenmiş bir veri kümesi elde etmeliyiz. 

## Veri kümesini yükleme ve temizleme

Veri kümesi olarak, [Twitter US Airline Sentiment](https://www.kaggle.com/crowdflower/twitter-airline-sentiment) veri kümesini kullanacağız.

In [ ]:
DATA_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART2/Day14-NLP/notebooks/datasets/"

DATASET_PATH = DATA_DIR + "twitter_airline_sentiment_tweets.csv"

In [ ]:
df = pd.read_csv(DATASET_PATH)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
sns.countplot(x = "airline_sentiment", data = df)

## Temel Ön İşlemler


In [ ]:
data = df[["airline_sentiment", "text"]]
data

In [ ]:
def sentiment(x):
    if x == 'positive':
        return 1
    elif x == 'negative':
        return -1
    else:
        return 0

In [ ]:
data["airline_sentiment_label"] = data["airline_sentiment"].apply(sentiment)
data

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stopwords = stopwords.words('english')

def clean_the_tweet(text):
  tokens= nltk.word_tokenize(re.sub("[^a-zA-Z]", " ",text))
  tokens = [token.lower() for token in tokens if token not in stopwords]
  return ' '.join(tokens[2:])

In [ ]:
print(f"Orignal Text : {data.text[11]}")
print()
print(f"Preprocessed Text : {clean_the_tweet(data.text[11])}")

In [ ]:
data["clean_text"] = data.text.map(clean_the_tweet)
data.head()

In [ ]:
data.iloc[10].text

In [ ]:
data.iloc[10].clean_text

In [ ]:
data

## Word2Vec

In [ ]:
%%time
documents = [_text.split() for _text in data.clean_text] 

In [ ]:
# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 16
W2V_MIN_COUNT = 10


w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)

In [ ]:
w2v_model.build_vocab(documents)

In [ ]:
vocab_len = len(w2v_model.wv.vocab)
print("Vocab size", vocab_len)

In [ ]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

In [ ]:
w2v_model.wv.most_similar("airlines")

In [ ]:
w2v_model.wv.vectors[0].shape[0]

In [ ]:
w2v_model.wv["american"].shape

In [ ]:
def get_mean_vector(sentence):
  tokens = nltk.word_tokenize(sentence)
  # remove out-of-vocabulary words
  words = [word for word in tokens if word in w2v_model.wv]
  if len(words) >= 1:
      return np.mean(w2v_model[words], axis=0)
  else:
      return np.zeros(300)


In [ ]:
content_vector = get_mean_vector(data.iloc[7].clean_text)
content_vector

In [ ]:
data["content_vector"] = data["clean_text"].apply(get_mean_vector)
data

In [ ]:
data.iloc[10].clean_text

In [ ]:
data.iloc[10]["content_vector"].shape

In [ ]:
X = np.zeros((14640, 300))

for i in range(len(data)):
  X[i] = data.iloc[i]["content_vector"]

X.shape

In [ ]:
y = data["airline_sentiment_label"].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
from sklearn import svm

SVM = svm.SVC(probability=True)

fit = SVM.fit(X_train, y_train)

In [ ]:
y_pred = fit.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_pred, y_test)
accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

cm = confusion_matrix(y_pred, y_test)

plt.figure()
plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
plt.xticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16,color='black')
plt.yticks(range(2), ['Negative', 'Neutral', 'Positive'], fontsize=16)
plt.show()